# Imports

In [ ]:
import os
# import pygsheets
import datetime
# from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import pandas_gbq
import json

# Constants

In [ ]:
ID = "FOLDER_ID" # full drive folder
# ID = "1O_jBUfR11bnDcRli1fEXBb4ZzLoGi7rC" # yandex pay folder - for test
LOG_PATH = "./"
TOKEN = f"{LOG_PATH}token.json"
CREDENTIALS = f"{LOG_PATH}credentials.json"
LOG_FILE_NAME = "drive_files_log.txt"

In [ ]:
FIELDS = [
    "kind",
    "driveId",
    "mimeType",
    "exportLinks",
    "parents",
    "lastModifyingUser",
    "owners",
    "webViewLink",
    "webContentLink",
    "size", 
    "id",
    "name",
    "description",
    "createdTime",
    "modifiedTime",
    "version",
    "originalFilename",
    "properties",
    "resourceKey",
]

FIELDS = ", ".join(FIELDS)
FIELDS


# Functions

In [ ]:
def google_auth():
    """
    Authenticate to Google Drive API and create a service object.
    
    This function uses OAuth credentials stored in the token.json file 
    to authenticate and create a Google Drive service object with read-only 
    access to drive metadata.

    Returns:
        service (Resource): Google Drive service object.
    """
    
    SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly"]

    creds = Credentials.from_authorized_user_file(TOKEN, SCOPES)
    service = build('drive', 'v3', credentials=creds)
    return service

In [ ]:
def get_request_data(auth: object, parent_id: str, time_filter: str = None) -> list:
    """
    Retrieve data about files in a specified directory within Google Drive.

    Args:
    auth: Authorized Google Drive service object.
    parent_id (str): The ID of the parent directory whose files are being queried.
    time_filter (str, optional): A filter based on modification time to narrow down the files. Defaults to None.

    Returns:
    list: A list of dictionaries where each dictionary contains metadata about each file.
    """

    if time_filter is None:
        files = []
        next_page_token = None
        while True:
            response = (
                auth.files()
                .list(
                    q=f"'{parent_id}' in parents",
                    spaces='drive',
                    fields=f"nextPageToken, files({FIELDS})",
                    pageToken=next_page_token,
                )
                .execute()
            )
            files_dict = response.get("files", [])
            files.extend([i for i in files_dict])
            next_page_token = response.get("nextPageToken")
            if next_page_token is None:
                break
    else:
        response = (
            auth.files()
            .list(
                q=f"'{parent_id}' in parents and modifiedTime > '{time_filter}'",
                fields=f"files({FIELDS})",
            )
            .execute()
        )
        files_dict = response.get("files", [])
        
    return files

In [ ]:
def get_files_dict(auth: object, drive_id: str) -> list:
    """
    Retrieve all files and sub-files metadata within specified Google Drive based on a given drive ID.

    Args:
        auth: Authorized Google Drive service object.
        drive_id (str): The ID of the Google Drive to retrieve files from.

    Returns:
        list: A list of dictionaries, where each dictionary contains metadata about a file or folder.
    """
    log_file = os.path.join(LOG_PATH, LOG_FILE_NAME)
    start_time = datetime.datetime.now()
    with open(log_file, "a") as log:
        log.write(f"Start Time: {start_time}\n")

    parent_folders = get_request_data(auth, ID)
    parents = [[i["id"], i["name"]] for i in parent_folders]
    print(parents)

    all_files = []
    count = 1
    for parent_data in parents:
        print(count, " ", parent_data[1])
        stack = get_request_data(auth, parent_data[0])
        
        while stack:
            current_item = stack.pop()
            current_item["offer_id"] = parent_data[0]
            current_item['offer_name'] = parent_data[1]
            all_files.append(current_item)

            if current_item["mimeType"] == "application/vnd.google-apps.folder":
                sub_items = get_request_data(
                    auth, current_item["id"])
                stack.extend([i for i in sub_items])
        count += 1

    with open(log_file, "a") as log:
        end_time = datetime.datetime.now()
        log.write(f"End Time: {end_time}\n")
        log.write(f"Process time: {end_time - start_time}\n")
        log.write(f"Total Number of Files: {len(all_files)}\n")
        for file in all_files:
            log.write(f'File Name: {file["name"]}, File ID: {file["id"]}\n')

    return all_files


In [ ]:
def corrections(df: pd.DataFrame, date_cols: list) -> pd.DataFrame:
    for col in date_cols:
        df[col] = pd.to_datetime(df[col]).dt.tz_localize(None)
    return df


In [ ]:
def format_to_json(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts DataFrame columns of type list and dict to JSON strings.

    Args:
        df (DataFrame): The DataFrame containing columns of type list or dict.

    Returns:
        DataFrame: The DataFrame with list and dict columns converted to JSON strings.
    """

    for col in df.columns:
        if isinstance(df[col][0], dict):
            df[col] = df[col].apply(json.dumps)

    for col in df.columns:
        if isinstance(df[col][0], list):
            df[col] = df[col].apply(json.dumps)

    return df

# Procсess

In [ ]:
# This script retrieves files metadata from Google Drive based on a given ID, 
# formats the metadata into a pandas DataFrame, 
# applies JSON formatting to list and dict columns, 
# corrects formatting issues (assumed to be done by `df_corrections()`), 
# and adds a timestamp of processing.

with open(LOG_PATH + LOG_FILE_NAME, "w") as log:
    log.write("")

files = get_files_dict(google_auth(), ID)
df_allcreo = pd.DataFrame(files)
df_allcreo = corrections(df_allcreo, ['createdTime', 'modifiedTime'])
df_allcreo = format_to_json(df_allcreo)

In [ ]:
info = json.loads(os.environ["BQ"])
creds = service_account.Credentials.from_service_account_info(info)

select_query = f"""
'SELECT id, max(modifiedTime) as modifiedTime_bd
FROM schema.table_with_creatives
GROUP BY id'
"""

# Execute the delete query using pandas-gbq
df_db = pd.io.gbq.read_gbq(select_query, project_id='project_id', credentials=creds)

In [ ]:
# Merge df_allcreo with df_db on 'id' column to combine data from Google Drive and the database, respectively
df_creo = pd.merge(df_allcreo, df_db, on="id", how="left")

"""
Filter the resulting DataFrame to include only rows where:
- The modification time in the database is older than the modification time from Google Drive (indicating an update),
- Or the file does not exist in the database (indicating a new upload).
"""

df_creo = df_creo[
    # creative was updated
    (df_creo["modifiedTime_bd"] < df_creo["modifiedTime"])
    # or new cretive was uploaded
    | (df_creo["modifiedTime_bd"].isna())
].drop(columns="modifiedTime_bd")

# Upload

In [ ]:
ids = df_creo['id'].tolist()

delete_query = f"""
DELETE FROM `schema`.`table_with_creatives`
WHERE id IN ({', '.join(['"{}"'.format(id) for id in ids])})
"""

# Execute the delete query using pandas-gbq
pd.io.gbq.read_gbq(delete_query, project_id='project_id', credentials=creds)

In [ ]:
pandas_gbq.to_gbq(
    df_creo,
    "schema.table_with_creatives",
    project_id="project_id",
    if_exists="append",
    credentials=creds,
    table_schema=[
        {"name": "createdTime", "type": "DATETIME"},
        {"name": "modifiedTime", "type": "DATETIME"},
    ],
)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0db8c4a9-831b-4e87-b54a-a26300df6861' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>